In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from matplotlib import pyplot as plt

from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr, ExtraTreesRegressor as etr
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

import xgboost as xgb


%matplotlib inline

In [1]:
#loading train
df_train=pd.read_csv('./final_train.csv')
#loading test
df_test=pd.read_csv('./final_test.csv')
y = pd.read_csv('/Users/Kenneth S. Hansen/Dropbox/MSc Business Intelligence/Kaggle/y.csv', header=None, index_col=0)

NameError: name 'pd' is not defined

In [24]:
print(len(y))
len(df_train)
y.head()

1458


,1
0,
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216


In [396]:
##Setting index as ID columns
df_test.set_index(['Id'], inplace = True)
df_train.set_index(['Id'], inplace = True)

In [397]:
##Setting y and x to begin running models
Y_vals = pd.DataFrame(df_train['SalePrice']).copy()
Y_vals = np.ravel(Y_vals)
df_test.drop(['Unnamed: 0', 'SalePrice'],inplace=True, axis =1)
df_train.drop(['Unnamed: 0', 'SalePrice'],inplace=True, axis =1)
X_train=df_train.copy()
X_test=df_test.copy()


# Lasso

In [44]:
##Grid search for optimal lasso parameters
grid_para_lasso = [{
    "n_alphas": range(100,1000, 100),
    "random_state": [0],
    "eps" : [0.0001],
    "normalize" : [True]
}]
grid_search_lasso = GridSearchCV(lasso_model, grid_para_lasso, cv=5, n_jobs=-1, verbose=1)
%time grid_search_lasso.fit(X_train, Y_vals)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.0min finished


CPU times: user 8.19 s, sys: 295 ms, total: 8.48 s
Wall time: 1min 2s


GridSearchCV(cv=5, error_score='raise',
       estimator=LassoCV(alphas=None, copy_X=True, cv=None, eps=0.0001, fit_intercept=True,
    max_iter=10000, n_alphas=100, n_jobs=1, normalize=True, positive=False,
    precompute='auto', random_state=0, selection='cyclic', tol=0.0001,
    verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_alphas': range(100, 1000, 100), 'random_state': [0], 'eps': [0.0001], 'normalize': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [498]:
print(grid_search_lasso.best_params_)
print(grid_search_lasso.best_score_)


NameError: name 'grid_search_lasso' is not defined

In [499]:
lasso_model = linear_model.LassoCV(random_state=0,n_alphas=1000, normalize=True, eps = 0.0001, max_iter=10000)
mod = lasso_model.fit(X_train, Y_vals)
mod.score(X_train, Y_vals)
##score=0.93082405005697799
#lasso_model.coef_

0.90827657658135419

# Elastic Net

In [370]:
##Grid search for optimal elastic parameters
grid_para_elastic = [{
    "n_alphas": [1000],
    #"alphas": np.linspace(0,20,num=5),
    "l1_ratio": np.linspace(0.1,1),
    "normalize" : [True]
}]
grid_search_elastic = GridSearchCV(elastic_model, grid_para_elastic, cv=5, n_jobs=-1, verbose=1)
%time grid_search_elastic.fit(X_train, Y_vals)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.4min finished


CPU times: user 14.9 s, sys: 737 ms, total: 15.6 s
Wall time: 4min 26s


GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.0001,
       fit_intercept=True, l1_ratio=0.5, max_iter=10000, n_alphas=1000,
       n_jobs=1, normalize=True, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_alphas': [1000], 'l1_ratio': array([ 0.1    ,  0.11837,  0.13673,  0.1551 ,  0.17347,  0.19184,
        0.2102 ,  0.22857,  0.24694,  0.26531,  0.28367,  0.30204,
        0.32041,  0.33878,  0.35714,  0.37551,  0.39388,  0.41224,
        0.43061,  0.44898,  0.46735,  0.48571,  0.5040... 0.8898 ,  0.90816,  0.92653,  0.9449 ,  0.96327,
        0.98163,  1.     ]), 'normalize': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [371]:
print(grid_search_elastic.best_params_)
print(grid_search_elastic.best_score_)

{'l1_ratio': 0.41224489795918373, 'n_alphas': 1000, 'normalize': True}
0.870019350982


In [387]:
elastic_model = linear_model.ElasticNetCV(l1_ratio=0.41224489795918373,n_alphas=1000, normalize=True,eps = 0.0001,max_iter=10000).fit(X_train, np.ravel(Y_vals))
mse = mean_squared_error(Y_vals, elastic_model.predict(X_train))
print(elastic_model.score(X_train, np.ravel(Y_vals)))
print(np.sqrt(mse))
#1-0.112947265448


0.905218654796
0.122935442528


# Random Forest Regressor

In [29]:
rfr_model = ensemble.RandomForestRegressor()

In [32]:
##Grid search for optimal random tree parameters
from sklearn import ensemble


from sklearn.model_selection import GridSearchCV
grid_para_rfr = [{
    "n_estimators": [1000],
    "max_depth": range(1, 6),
    "min_samples_leaf": range(1,6),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [0]
}]

grid_search_rfr = GridSearchCV(rfr_model, grid_para_rfr, cv=10, n_jobs=-1, verbose=1, scoring='neg_mean_squared_error')
%time grid_search_rfr.fit(df_train, np.ravel(y))

Fitting 10 folds for each of 125 candidates, totalling 1250 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 31.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 61.5min
[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 62.0min finished


Wall time: 1h 2min 12s


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [1000], 'max_depth': range(1, 6), 'min_samples_leaf': range(1, 6), 'min_samples_split': array([ 2,  9, 16, 23, 30]), 'random_state': [0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [25]:
##Grid search for optimal random tree parameters
#Agreesive
from sklearn import ensemble


from sklearn.model_selection import GridSearchCV
grid_para_rfr = [{
    "n_estimators": [1000],
    "max_depth": range(7,9),
    "min_samples_leaf": range(2, 5),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [0]
}]

grid_search_rfr = GridSearchCV(rfr_model, grid_para_rfr, cv=5, n_jobs=-1, verbose=1)
%time grid_search_rfr.fit(df_train, np.ravel(y))

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed: 20.2min finished


Wall time: 20min 32s


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [1000], 'max_depth': range(8, 11), 'min_samples_leaf': range(2, 5), 'min_samples_split': array([ 2,  9, 16, 23, 30]), 'random_state': [0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [33]:
print(grid_search_rfr.best_params_)
print(grid_search_rfr.best_score_)
#0.88759734575697047
#{'max_depth': 9,
# 'min_samples_leaf': 1,
# 'min_samples_split': 2,
# 'n_estimators': 1000,
# 'random_state': 0}

{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 0}
-0.000193868552163


In [453]:
rfr_model = rfr(random_state=0, min_samples_leaf = 2, min_samples_split =2,
        n_estimators=1000, max_depth=9,  max_features='sqrt', oob_score=True).fit(X_train, np.ravel(Y_vals))
print(rfr_model.score(X_train, np.ravel(Y_vals)))
print(rfr_model.oob_score_)

0.927213532396
0.853760776452


# Extra Random Trees

In [27]:
##Grid search for extra random tree parameters
from sklearn.model_selection import GridSearchCV
grid_para_etr = [{
    "n_estimators": [1000],
    "max_depth": range(1,10),
    "min_samples_leaf": range(1, 5),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [0]
}]
grid_search_etr = GridSearchCV(etr_model, grid_para_etr, cv=5, n_jobs=-1, verbose=1)
%time grid_search_etr.fit(df_train, np.ravel(y))

NameError: name 'etr_model' is not defined

In [455]:
print(grid_search_etr.best_params_)
print(grid_search_etr.best_score_)
#{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 0}
#0.81466266301

{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 0}
0.809695308898


In [456]:
etr_model = etr(random_state=0,min_samples_split=2,
        n_estimators=1000, max_depth=9, min_samples_leaf= 1, max_features='sqrt', bootstrap=True,oob_score=True).fit(X_train,Y_vals)
print(etr_model.score(X_train, Y_vals))
print(etr_model.oob_score_)

0.899224010175
0.811302222087


# XGB

In [ ]:
#Grid Search for XGB -- Optimized via Ilyas Bayesion Optimizer instead
grid_para_xgb = [{
    'max_depth': [3,4,5,6,7],
    'learning_rate': [.001, .01, .1],
    'n_estimators': [200],
    'gamma': [0.1],
    'reg_alpha': [.25, .5, .75],
    'reg_lambda': [.25, .5, .75],
    'subsample': [.3 ,.5,.7,.9],
    'colsample_bytree': [.4,.5,.6],
    'min_child_weight': [.3, .6, .9]
}]
grid_search_xgb = GridSearchCV(xgb_model, grid_para_etr, cv=5, n_jobs=-1, verbose=1)
%time grid_search_xgb.fit(X_train, Y_vals)

In [434]:
xgb_model = xgb.XGBRegressor(
                 colsample_bytree=1,
                 gamma=0.0,
                 learning_rate=0.05,
                 max_depth=20,
                 min_child_weight=0.1,
                 n_estimators=1000,                                                                  
                 reg_alpha=0,
                 reg_lambda=1,
                 subsample=1,
                 seed=1,
                 silent=False)

xgb_model.fit(X_train, Y_vals)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.0, learning_rate=0.05, max_delta_step=0, max_depth=20,
       min_child_weight=0.1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1, silent=False, subsample=1)

In [435]:
xgb_model.score(X_train, Y_vals)

0.99999792129154841

# LGBM

In [429]:
##Should tune this algorithm, but not sure what any of the parameters do

lgb_model = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
lgb_model.fit(X_train,Y_vals)


LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', colsample_bytree=1.0, feature_fraction=0.2319,
       feature_fraction_seed=9, learning_rate=0.05, max_bin=55,
       max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_data_in_leaf=6, min_split_gain=0.0, min_sum_hessian_in_leaf=11,
       n_estimators=720, n_jobs=-1, num_leaves=5, objective='regression',
       random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=50000, subsample_freq=1)

In [430]:
lgb_model.score(X_train, Y_vals)

0.96393645930842231

# Results

In [483]:
n_folds = 4

scorer = make_scorer(mean_squared_error,greater_is_better = False)
def rmse_CV_train(model):
    kf = KFold(n_folds,shuffle=True,random_state=42).get_n_splits(X_train.values)
    rmse = np.sqrt(-cross_val_score(model,X_train,Y_vals,scoring ="neg_mean_squared_error",cv=kf, n_jobs=-1, verbose=1))
    return (rmse)
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))



In [512]:
models = [lasso_model, elastic_model, xgb_model, rfr_model, etr_model, lgb_model]
for i in models:
    score = rmsle(Y_vals, i.predict(X_train))
    print('rmse for '+str(i)[0:7]+ ": ",score)
   

rmse for LassoCV:  0.0101191420426
rmse for Elastic:  0.0102803446154
rmse for XGBRegr:  4.78871005635e-05
rmse for RandomF:  0.00908996582226
rmse for ExtraTr:  0.0106191419038
rmse for LGBMReg:  0.00640292114589


# RMSE Score on each model

In [497]:
models = [lasso_model, elastic_model, rfr_model, etr_model, lgb_model]
for i in models:
    score = rmse_CV_train(i)
    print('rmse mean/std for '+str(i)[0:7]+ ": ",score.mean(), score.std())
   

rmse mean/std for LassoCV:  0.144388099798 0.0231283159032


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.9s finished


rmse mean/std for Elastic:  0.142590646106 0.0197249454264


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    5.5s finished


rmse mean/std for RandomF:  0.155135199282 0.0138732019442


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.6s finished


rmse mean/std for ExtraTr:  0.174802628431 0.0132328851953


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.4s finished


rmse mean/std for LGBMReg:  0.124189119792 0.0132627221085


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    3.6s finished


# Saving results to CSV

In [458]:
for i in models:
    results=round(pd.DataFrame(np.exp(i.predict(X_test))),3)
    results.columns=['SalePrice']
    results.set_index(df_test.index, inplace=True)
    results.to_csv('output_'+str(i)[0:7]+'.csv')

In [459]:
df_etr=pd.read_csv('./output_ExtraTr.csv', index_col='Id')
df_rfr=pd.read_csv('./output_RandomF.csv', index_col='Id')
df_lasso=pd.read_csv('./output_LassoCV.csv', index_col='Id')
df_elastic=pd.read_csv('./output_Elastic.csv', index_col='Id')
df_xgb=pd.read_csv('./output_XGBRegr.csv', index_col='Id')
df_lgb=pd.read_csv('./output_LGBMReg.csv', index_col='Id')



# Averaged Model

In [505]:
df_average = (df_rfr+df_lasso+df_elastic+df_etr+df_xgb+df_lgb)/6
df_average.head()

,SalePrice
Id,
1461,123845.698729
1462,156090.811958
1463,184959.109250
1464,192666.678500
1465,193633.896208


In [506]:
df_average.to_csv('ALL_average.csv')

# Stacking

In [462]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [507]:
stacked_averaged_models = StackingAveragedModels(base_models = (elastic_model, rfr_model, etr_model),
                                                 meta_model = lasso_model)




In [508]:
stacked_averaged_models.fit(X_train.values, Y_vals)


StackingAveragedModels(base_models=(ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.0001,
       fit_intercept=True, l1_ratio=0.41224489795918373, max_iter=10000,
       n_alphas=1000, n_jobs=1, normalize=True, positive=False,
       precompute='auto', random_state=None, selection='cyclic',
       tol=0.0001, verbos...n_estimators=1000, n_jobs=1,
          oob_score=True, random_state=0, verbose=0, warm_start=False)),
            meta_model=LassoCV(alphas=None, copy_X=True, cv=None, eps=0.0001, fit_intercept=True,
    max_iter=10000, n_alphas=1000, n_jobs=1, normalize=True,
    positive=False, precompute='auto', random_state=0, selection='cyclic',
    tol=0.0001, verbose=False),
            n_folds=5)

In [509]:
stacked_pred = np.exp(stacked_averaged_models.predict(X_test.values))
xgb_pred=np.exp(xgb_model.predict(X_test))
lgb_pred=np.exp(lgb_model.predict(X_test))
ensemble= stacked_pred*0.7 + xgb_pred *0.15 + lgb_pred*0.15



In [510]:
sub = pd.DataFrame()
sub['SalePrice'] = np.round(ensemble,3)
sub.set_index(df_test.index, inplace=True)
sub.to_csv('submission.csv')

In [352]:
stacked_results=round(pd.DataFrame(np.exp(stacked_averaged_models.predict(X_test.values))),3)
stacked_results.columns=['SalePrice']
stacked_results.set_index(df_test.index, inplace=True)
stacked_results.to_csv('stacked_model.csv')

In [511]:
sub.head()

,SalePrice
Id,
1461,121045.564
1462,156573.502
1463,185822.156
1464,194683.013
1465,194890.822


In [484]:
from stacking import stacking_regression


In [486]:
meta_model = lasso_model

In [487]:
stacking_prediction = stacking_regression(models, meta_model, X_train, Y_vals, X_test,
                              transform_pred = np.expm1, 
                               metric=rmsle, verbose=1)

metric: [rmsle]

model 0: [LassoCV]
    ----
    MEAN:   [9.54594010]

model 1: [ElasticNetCV]
    ----
    MEAN:   [9.54454733]

model 2: [RandomForestRegressor]
    ----
    MEAN:   [9.54234048]

model 3: [ExtraTreesRegressor]
    ----
    MEAN:   [9.54071370]

model 4: [XGBRegressor]
    ----
    MEAN:   [9.54330248]

model 5: [LGBMRegressor]
    ----
    MEAN:   [9.54483468]



In [488]:
stacking_prediction

array([ 125872.85637041,  150108.37848661,  178119.52683864, ...,
        157552.96526958,  122456.88939299,  207038.61891604])

In [495]:
sp=round(pd.DataFrame(stacking_prediction),3)
sp.set_index(df_test.index, inplace=True)
sp.columns = ['SalePrice']
sp.to_csv('ZYrun.csv')
sp

,SalePrice
Id,
1461,125872.856
1462,150108.378
1463,178119.527
1464,185903.661
1465,184917.702
1466,166934.519
1467,160811.239
1468,161489.799
1469,168804.036


In [10]:
#loading train
df_train=pd.read_csv('./final_train.csv')
#loading test
df_test=pd.read_csv('./final_test.csv')
y = pd.read_csv('/Users/Kenneth S. Hansen/Dropbox/MSc Business Intelligence/Kaggle/y.csv', header=None, index_col=1)

In [236]:
df = pd.read_csv('/Users/Kenneth S. Hansen/Dropbox/MSc Business Intelligence/Kaggle/df_all.csv', index_col=0)
df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,TotLivArea,TotalBath,BsmtUnFinRatio,AreaPerCar,AvgRoomSize,GarageScore,OverallGrade,Overallscore,ExterGrade,AllPorchSF
0,856,854,0,1,3,1Fam,3,1,706.0,0.0,...,2566.0,3.5,0.175234,274.000000,213.750000,AttchdAttchdAttchd,35,59850,12,61
1,1262,0,0,1,3,1Fam,3,4,978.0,0.0,...,2524.0,2.5,0.225040,230.000000,210.333333,AttchdAttchdAttchd,48,60576,9,0
2,920,866,0,1,3,1Fam,3,2,486.0,0.0,...,2706.0,3.5,0.471739,304.000000,297.666667,AttchdAttchdAttchd,35,62510,12,42
3,961,756,0,1,3,1Fam,4,1,216.0,0.0,...,2473.0,2.0,0.714286,214.000000,245.285714,DetchdDetchdDetchd,35,60095,9,307
4,1145,1053,0,1,4,1Fam,3,3,655.0,0.0,...,3343.0,3.5,0.427948,278.666667,244.222222,AttchdAttchdAttchd,40,87920,12,84


In [28]:
from scipy.stats import skew, kurtosis
from scipy.special import boxcox1p

In [238]:
numerical_features = list(df.select_dtypes(exclude = ['object']).columns)

In [239]:
skewness = df[numerical_features].apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.7]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
print(skewness)

43 skewed numerical features to log transform
1stFlrSF          1.257286
2ndFlrSF          0.861556
3SsnPorch        11.372080
BsmtCond         -3.602661
BsmtExposure      1.119066
BsmtFinSF1        0.980645
BsmtFinSF2        4.144503
BsmtFinType2      3.150951
BsmtHalfBath      3.929996
BsmtQual         -1.271611
BsmtUnfSF         0.919688
CentralAir       -3.457555
EnclosedPorch     4.002344
ExterCond         1.315069
ExterQual         0.783456
Fence             1.753731
Fireplaces        0.725278
Functional       -4.961675
GarageCond       -3.381673
GarageQual       -3.262260
GarageYrBlt      -3.904632
GrLivArea         1.068750
KitchenAbvGr      4.300550
LandSlope        -4.973254
LotArea          13.109495
LotFrontage       1.103039
LotShape         -1.247973
LowQualFinSF     12.084539
MasVnrArea        2.621719
MiscVal          21.939672
OpenPorchSF       2.529358
PavedDrive       -2.977741
PoolArea         17.688664
PoolQC           20.341424
ScreenPorch       3.945101
Street   

In [240]:
skewness['3SsnPorch']
df['3SsnPorch']

0         0
1         0
2         0
3         0
4         0
5       320
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
       ... 
1429      0
1430      0
1431      0
1432      0
1433      0
1434      0
1435      0
1436      0
1437      0
1438      0
1439      0
1440      0
1441      0
1442      0
1443      0
1444      0
1445      0
1446      0
1447      0
1448      0
1449      0
1450      0
1451      0
1452      0
1453      0
1454      0
1455      0
1456      0
1457      0
1458      0
Name: 3SsnPorch, Length: 2917, dtype: int64

In [241]:
lambda_ = 0.001
skew_list = ['3SsnPorch', 'LotArea', 'LowQualFinSF', 'MiscVal', 'PoolArea', 
             'PoolQC', 'Street']

for x in skew_list:
    df[x] = boxcox1p(df[x], lambda_)

In [230]:
##3SsnPorch
from scipy.special import boxcox1p
lam = 0.001
print(str(skewness['3SsnPorch']) + " Skewness BERFORE transformation")
x = boxcox1p(skewness['3SsnPorch'], lam)
print(str(x) + " Skewnes AFTER transformation")

11.3720799331 Skewness BERFORE transformation
2.51860869502 Skewnes AFTER transformation


In [175]:
##LotArea
lam = 0.001
print(str(skewness['LotArea']) + " Skewness BERFORE transformation")
xx = boxcox1p(skewness['LotArea'], lam)
print(str(xx) + " Skewnes AFTER transformation")

13.1094946932 Skewness BERFORE transformation
2.6503539479 Skewnes AFTER transformation


In [182]:
#LowQualFinSF
lam = 0.001
print(str(skewness['LowQualFinSF']) + " Skewness BERFORE transformation")
xxx = boxcox1p(skewness['LowQualFinSF'], lam)
print(str(xxx) + " Skewnes AFTER transformation")

12.0845387954 Skewness BERFORE transformation
2.57474025363 Skewnes AFTER transformation


In [193]:
#MiscVal
lam = 0.001
print(str(skewness['MiscVal']) + " Skewness BERFORE transformation")
y = boxcox1p(skewness['MiscVal'], lam)
print(str(y) + " Skewnes AFTER transformation")

21.9396721704 Skewness BERFORE transformation
3.13778038004 Skewnes AFTER transformation


In [198]:
#PoolArea
lam = 0.001
print(str(skewness['PoolArea']) + " Skewness BERFORE transformation")
yy = boxcox1p(skewness['PoolArea'], lam)
print(str(yy) + " Skewnes AFTER transformation")

17.688664487 Skewness BERFORE transformation
2.93220769877 Skewnes AFTER transformation


In [210]:
#PoolQC  
lam = 0.001
print(str(skewness['PoolQC']) + " Skewness BERFORE transformation")
yyy = boxcox1p(skewness['PoolQC'], lam)
print(str(yyy) + " Skewnes AFTER transformation")

20.3414240859 Skewness BERFORE transformation
3.0653385481 Skewnes AFTER transformation


In [206]:
#Street
lam = 0.001
print(str(skewness['Street']) + " Skewness BERFORE transformation")
a = boxcox1p(skewness['Street'], lam)
print(str(a) + " Skewnes AFTER transformation")

-15.4947560207 Skewness BERFORE transformation
nan Skewnes AFTER transformation


In [244]:
#ScreenPorch
lam = 0.001
print(str(skewness['ScreenPorch']) + " Skewness BERFORE transformation")
yyy = boxcox1p(skewness['ScreenPorch'], lam)
print(str(yyy) + " Skewnes AFTER transformation")

3.94510122562 Skewness BERFORE transformation
1.59967555323 Skewnes AFTER transformation
